# Laborarbeit KI - CSP
**Belegung von Laboren für Projektpräsentationen**

Zuerste müssen alle benötigten Bibliotheken installiert werden

In [6]:
!pip install -r requirements.txt

Jetzt werden alle benötigten Bibliotheken importiert und ein Datensatz als DataFrame geladen. Um einen ersten Überblick zu bekommen.

In [7]:
from constraint import *
import pandas as pd

problem1 = pd.read_csv('./data/DS_CSP_1/pr_conf_001.txt', sep=';')
problem1

,Studiengang,Projektgruppen,Kommissionen
0,Informatik,8,3
1,Elektrotechnik,4,2
2,WIW,1,3
3,Maschinenbau,5,2
4,Data Science,2,1


Wir haben in diesem Datensatz 5 verschiedene Studiengänge und dabei hat Informatik die meisten Projektgruppen.

## Umgebung definieren
Jetzt werden die Umgebungsvariablen definiert (Räume, Solts, ...).

In [8]:
DAYS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
SLOTS = ['S1', 'S2', 'S3', 'S4']
SLOT_TIMES = {'S1': '08:00 - 09:00', 'S2': '10:00 - 11:00', 'S3': '13:00 - 14:00', 'S4': '15:00 - 16:00'}
ROOMS = ['L1', 'L2', 'L3']
AMOUNT_PRESENTATIONS = 3
MAX_SWICHTES = 1

MAX_SLOTS = len(DAYS) * len(SLOTS) * len(ROOMS)
MAX_SLOTS

60

Es gibt insgesamt 60 Slots, verteilt über drei Räume von Mo. - Fr.

## Hilfsfunktionen
Hier werden zuerst Hilfsfunktionen definert, um die spätere Problem-Funktion nicht zu überfüllen.
- `getDomain()`: Erstellt die Domain für den jeweiligen Studiengang
- `getVariables()`: Erstellt für den jeweiligen Studiengang eine Liste von allen Gruppen, mit ihren 3 Präsentationen

In [34]:
def getDomain(study_course: str, amount_committees: int):
    """
    Create the domian for the presentation variables, based on days, slots, rooms and committees.
    
    Returns: a list of tuples (day, slot, room, committee)
    """
    domain = []
    for day in DAYS:
        for slot in SLOTS:
            for room in ROOMS:
                for committee in range(amount_committees):
                    domain.append((day, slot, room, f"{study_course.replace(' ', '_')}_K{committee}"))
    return domain

def getVariables(study_course: str, amount_groups: int):
    """
    Defines all variables for a given study course and amount of groups.
    
    Returns: a list of variable names
    """
    variables = []

    for group in range(1, amount_groups + 1):
        for p in range(1, AMOUNT_PRESENTATIONS + 1):
            var_name = f"{study_course.replace(' ', '_')}_g{group}_P{p}"  # Unique name - e.g. "Informatikg1_P1"
            variables.append(var_name)
    return variables

## Constraints
Jetzt müssen die jeweiligen constraints erstellt werden. Dafür wird hier jedes Constraint als Funktion definiert und später zum Projekt hinzugefügt.

**Constraints**:
- `unique_appointment`: Jede Präsentation braucht einen anderen Termin (Tag / Zeit)
- `group_clash`: Eine Gruppe kann nicht mehrere Präsentationen halten in einem Zeitslot
- `comitee_clash`: Eine Komission kann nicht mehreren Präsenationen gleichzeitig zuhören
- `max_switches`: Jede Komission darf **max** einmal am Tag den Raum wechseln
- `rest_time`: Wenn eine Gruppe den letzten Slot hat, darf sie nicht den ersten Slot am direkt nächsten Tag haben
- `early_appointment`: Präsentationen sollen bevorzugt in frühen Slots stattfinden (_weich_).

In [ ]:
def unique_appointment():
    pass

def group_clash():
    pass

def committee_clash():
    pass

def max_switches():
    pass

def rest_time():
    pass

def early_appointment():
    pass

## Problem definieren
Hier wird eine Funktion definiert, welche das Problem repräsentiert. Dabei werden die Variablen geladen und die Constraints hinzugefügt, welche zuvor definiert wurden.

Zuerst werden für jede Präsentation von jeder Gruppe eine Varibale hinzugefügt, mit der Domäne `(Tag, Slot, Raum, Komissionen)`. 

In [ ]:
def presentation_problem(input_data: pd.DataFrame, solver: str = 'backtracking'):
    problem = Problem()
    
    # --- Define variables ---
    variables = []
    for study in input_data.itertuples():
        print(study)
        vars = getVariables(study[1], study[3])
        domain = getDomain(study[1], study[2])
        variables.extend(vars)
        
        problem.addVariables(vars, domain)
    
    
    # --- little check ---
    if len(variables) > MAX_SLOTS:
        print("Too many variables for available slots!")
        return None
    elif len(variables) == MAX_SLOTS:
        print("Warning: Variables equal to available slots, no freedom in scheduling.")
    
    
    # --- Define Constraints ---
    
        
presentation_problem(problem1, solver='backtracking')